In [93]:
import pandas as pd
import requests


<pre>

Variables(12):

1. Real GDP Growth✓ 
2. Interest Rate ✓ 
3. Federal Budget Deficit (% GDP) ✓
4. Debt Held by the Public (% GDP) ✓
5. Primary Budget Deficit (% GDP)
6. Federal Revenue Growth (YoY)
7. Federal Spending Growth (YoY)
8. Effective Federal Funds Rate
9. 10-Year Treasury Yield
10. Inflation Rate (CPI YoY)
11. Unemployment Rate
12. Real GDP per Capita
13. Population Age 65+ (%)

</pre>




In [107]:
#api key
key = 
#endpoint and parameters
endpoint = 'https://api.stlouisfed.org/fred/series/observations'


**DataFrame for Interest Rates**

In [149]:
params_1 = {
    'series_id' : 'FEDFUNDS',
    'api_key' : key,
    'file_type' : 'json',
    'observation_start' : '2000-01-01',
    'observation_end' : '2025-01-01'
}

response_1 = requests.get(endpoint, params=params_1)
data_1 = response_1.json()
observations = data_1['observations']
df = pd.DataFrame(observations)
df['date'] = pd.to_datetime(df['date'])
df['value'] = pd.to_numeric(df['value'], errors='coerce')  # coerce invalid entries to NaN
df.set_index('date', inplace=True)

# Resample only the numeric 'value' column
df_annual = df[['value']].resample('YE').mean()


**DataFrame for GDP**

In [183]:
params= {
    'series_id': 'GDP',
    'api_key': key,
    'file_type': 'json',
    'observation_start': '1999-01-01',
    'observation_end': '2024-12-31'
}

response = requests.get(endpoint,params)
data = response.json()
df_2 = pd.DataFrame(data['observations'])
df_2['date'] = pd.to_datetime(df_2['date'])
df_2['value'] = pd.to_numeric(df_2['value'], errors='coerce')  # Handle any "." values
df_2.set_index('date', inplace=True)

# Convert quarterly GDP to annual GDP (use .sum() for total annual GDP)
df_annual = df_2[['value']].resample('YE').sum()  # or .mean() if you prefer average

# Step 1: Initialize an empty list
pct_changes = []

# Step 2: Convert index to year for cleaner labeling (optional)
df_annual.index = df_annual.index.year

# Step 3: Loop through and compute % change
years = df_annual.index.tolist()
values = df_annual['value'].tolist()

for i in range(1, len(values)):
    prev = values[i - 1]
    curr = values[i]
    if prev != 0:
        pct_change = ((curr - prev) / prev) * 100
    else:
        pct_change = None  # or np.nan to avoid division by zero
    pct_changes.append((years[i], pct_change))  # Skip year 0 because no prior data

# Step 4: Turn into DataFrame
df_pct_change = pd.DataFrame(pct_changes, columns=['Year', 'YoY_Percent_Change'])


,Year,YoY_Percent_Change
0,2000,6.435149
1,2001,3.228744
2,2002,3.280869
3,2003,4.825108
4,2004,6.640332
5,2005,6.728232
6,2006,5.954247
7,2007,4.767399
8,2008,2.042491
9,2009,-1.975608


**Percentage Change of Federal Budget Deficit**
<pre>
if negative, means that government used more money then it had
</pre>

In [187]:
params_3 = {
    'series_id': 'MTSDS133FMS',
    'api_key': key,
    'file_type': 'json',
    'observation_start': '1999-01-01',
    'observation_end': '2024-12-31'
}

response_deficit = requests.get(endpoint,params_3)
data_deficit = response_deficit.json()

df_deficit = pd.DataFrame(data_deficit['observations'])
df_deficit['date'] = pd.to_datetime(df_deficit['date'])
df_deficit['value'] = pd.to_numeric(df_deficit['value'], errors='coerce')
df_deficit.set_index('date', inplace=True)

df_GDP_Deficit = df_deficit[['value']].resample('YE').sum()

# Step 1: Initialize an empty list
pct_changes_gdp = []

# Step 2: Convert index to year for cleaner labeling (optional)
df_GDP_Deficit.index = df_GDP_Deficit.index.year

# Step 3: Loop through and compute % change
years = df_GDP_Deficit.index.tolist()
values = df_GDP_Deficit['value'].tolist()

for i in range(1, len(values)):
    prev = values[i - 1]
    curr = values[i]
    if prev != 0:
        pct_change_gdp = ((curr - prev) / prev) * 100
    else:
        pct_change_gdp = None  # or np.nan to avoid division by zero
    pct_changes_gdp.append((years[i], pct_change_gdp))  # Skip year 0 because no prior data

# Step 4: Turn into DataFrame
df_pct_change_gdp = pd.DataFrame(pct_changes_gdp, columns=['Year', 'YoY_Percent_Change'])

**Percentage of Debt held by the people**

In [188]:
params_3 = {
    'series_id': 'FYGFDPUN',
    'api_key': key,
    'file_type': 'json',
    'observation_start': '1999-01-01',
    'observation_end': '2024-12-31'
}

response_debtHeldbyPeople = requests.get(endpoint,params_3)
data_debtHeldbyPeople = response_debtHeldbyPeople.json()

df_debtbyPeople = pd.DataFrame(data_debtHeldbyPeople['observations'])
df_debtbyPeople['date'] = pd.to_datetime(df_debtbyPeople['date'])
df_debtbyPeople['value'] = pd.to_numeric(df_debtbyPeople['value'], errors='coerce')
df_debtbyPeople.set_index('date', inplace=True)

df_debtHeldbyPeople = df_debtbyPeople[['value']].resample('YE').last()

# Step 1: Initialize an empty list
debtHeldbyPeople_changes = []

# Step 2: Convert index to year for cleaner labeling (optional)
df_debtHeldbyPeople.index = df_debtHeldbyPeople.index.year

# Step 3: Loop through and compute % change
years = df_debtHeldbyPeople.index.tolist()
values = df_debtHeldbyPeople['value'].tolist()

for i in range(1, len(values)):
    prev = values[i - 1]
    curr = values[i]
    if prev != 0:
        debtHeldbyPeople_change = ((curr - prev) / prev) * 100
    else:
        debtHeldbyPeople_changes = None  # or np.nan to avoid division by zero
    debtHeldbyPeople_changes.append((years[i], debtHeldbyPeople_change))  # Skip year 0 because no prior data

# Step 4: Turn into DataFrame
df_debtHeldbyPeople_changes = pd.DataFrame(debtHeldbyPeople_changes, columns=['Year', 'YoY_Percent_Change'])
